# Evaluation for Assistant API

Dataset chosen is the famous `hotspotqa` which is commonly used to evaluate QA and context understanding. And only hard level questions in [validation split](https://huggingface.co/datasets/scholarly-shadows-syndicate/hotpotqa_with_qa_gpt35/viewer/default/validation) is used in this notebook. 



In [6]:
!pip install datasets

  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 418.6 kB/s eta 0:00:00a 0:00:01
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 785.4 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 658.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 30.5 kB/s eta 0:00:0000:0100:35m
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
   ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/11.3 MB 61.2 kB/s eta 0:01:52^C
   ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/11.3 MB 61.5 kB/s eta 0:01:51
ERROR: Operation cancelle

In [4]:
%reload_ext autoreload
%autoreload 2

In [17]:
from datasets import load_dataset

dataset = load_dataset("scholarly-shadows-syndicate/hotpotqa_with_qa_gpt35")
dataset["validation"][0]



{'id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'type': 'comparison',
 'level': 'hard',
 'supporting_facts': {'title': ['Scott Derrickson', 'Ed Wood'],
  'sent_id': [0, 0]},
 'context': {'title': ['Ed Wood (film)',
   'Scott Derrickson',
   'Woodson, Arkansas',
   'Tyler Bates',
   'Ed Wood',
   'Deliver Us from Evil (2014 film)',
   'Adam Collis',
   'Sinister (film)',
   'Conrad Brooks',
   'Doctor Strange (2016 film)'],
  'sentences': [['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.'],
   ['Scott Derrickson (born Jul

Start mini assistant server.

* `llm_compiler` is used for agent execution
* `mixtral 7bx8` hosted by vLLM is used as model provider 

```shell
mini-assistant --db_file_path /tmp/assistant_eval.db \
   --file_store_path /tmp/mini-assistant-files \
   --agent-executor-type llm_compiler \
   --verbose
```

In [16]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:9000/v1")

client.beta.assistants.create(name="hotpotqa-bot")

with open("output/hotqa_result.json", "wb") as output_json:
    client.beta.threads.create_and_run_poll()
    
    

{'id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'type': 'comparison',
 'level': 'hard',
 'supporting_facts': {'title': ['Scott Derrickson', 'Ed Wood'],
  'sent_id': [0, 0]},
 'context': {'title': ['Ed Wood (film)',
   'Scott Derrickson',
   'Woodson, Arkansas',
   'Tyler Bates',
   'Ed Wood',
   'Deliver Us from Evil (2014 film)',
   'Adam Collis',
   'Sinister (film)',
   'Conrad Brooks',
   'Doctor Strange (2016 film)'],
  'sentences': [['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.'],
   ['Scott Derrickson (born Jul

In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def compare_answer(answer: str, label: str):
    """Compare the answer (from Agent) and label (GT).
    Label can be either a string or a number.
    If label is a number, we allow 10% margin.
    Otherwise, we do the best-effort string matching.
    """
    if answer is None:
        return False

    # see if label is a number, e.g. "1.0" or "1"
    if is_number(label):
        label = float(label)
        # try cast answer to float and return false if it fails
        try:
            answer = float(answer)
        except:
            return False
        # allow 10% margin
        if answer > label * 0.9 and answer < label * 1.1:
            return True
        else:
            return False

    else:
        label = normalize_answer(label)
        answer = normalize_answer(answer)
        return answer == label
